In [1]:
import os
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
df = pd.read_csv("/content/drive/MyDrive/sujet_images_by_class/finance_data.csv")

In [17]:
#encode the labels
le = LabelEncoder()
le.fit_transform(df["label"])
# df["label"] = le.fit_transform(df["label"])

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
df.label.nunique() #unique labels

8

In [ ]:
#convert to hugging face dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

In [4]:
#tokenization

from transformers import AutoTokenizer

In [5]:
model_checkpoint = "yiyanghkust/finbert-pretrain"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], max_length=512, padding=True, truncation=True)

In [ ]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

Define the model

In [6]:
from transformers import AutoModelForSequenceClassification

In [ ]:
num_labels = df["label"].nunique()

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Loop

In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_score": f1_score(labels, preds, average="weighted")
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models/finbert_classifier",
    eval_strategy= "epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/models/logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_score",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,No log,0.652286,0.901961,0.899597
2,No log,0.508704,0.921569,0.922033
3,No log,0.581060,0.911765,0.910796
4,No log,0.560175,0.911765,0.910796
5,No log,0.551338,0.901961,0.899609
6,No log,0.550552,0.901961,0.899609
7,No log,0.550059,0.901961,0.899609
8,No log,0.549752,0.901961,0.899609
9,No log,0.550412,0.901961,0.899609
10,No log,0.550466,0.901961,0.899609


TrainOutput(global_step=260, training_loss=0.009035954108605018, metrics={'train_runtime': 556.1732, 'train_samples_per_second': 7.318, 'train_steps_per_second': 0.467, 'total_flos': 1163539765149696.0, 'train_loss': 0.009035954108605018, 'epoch': 10.0})

In [ ]:
# help(TrainingArguments)

In [ ]:
trainer.save_model("/content/drive/MyDrive/models/financial_finbert_classfier")

Test

In [7]:
import torch
import torch.nn.functional as F

In [8]:
path = "/content/drive/MyDrive/models/financial_finbert_classfier"

model = AutoModelForSequenceClassification.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")

In [9]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
text = "INVOICE International Research and Devclopmert Corporation No..6807 MALTAWAK, MICHIGAN 49071 \
ARACODE \
CEE-E99 ENOMTEL \
Dr.Thomas S. Osdene \
Director of Research \
1O- \
Phillip Morris Research Center \
PO.Box 26583 \
RichmondVA23261 \
February 23,1978 \
SIUOYN \
368-001/368-010 \
OURP.O.NO \
TERMSPAYABLE UPON RECEIPT \
For Professional Services \
$10,000.00 \
Acute Oral Toxicity Study in Rats \
Less Amount Invoiced on October 25.1977 \
$2490.00 \
3,960.00 \
$1,470.00 \
1.470.00 \
$8530.00 \
2001211506"

In [11]:
#tokenize input

inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)

In [12]:
#inference
with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits
  probs = F.softmax(logits, dim=-1)
  predicted_class = torch.argmax(probs, dim=-1)

In [13]:
#print result
print(f"Predicted class: {predicted_class}")
print(f"Class probabilites: {probs}")

Predicted class: tensor([6])
Class probabilites: tensor([[3.8750e-05, 9.1169e-05, 3.2855e-05, 4.0354e-05, 3.3268e-05, 8.4815e-05,
         9.9964e-01, 4.3062e-05]])


In [15]:
# name = le.inverse_transform
int(predicted_class[0])

6

In [22]:
le.inverse_transform([int(predicted_class[0])])[0]

'Project Documents'